In [ ]:
import pandas as pd
import pyemu
import os, glob
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
sns.set()

In [ ]:
wkdir = '../run_data/'
pstroot='prior_mc_wide'
noptmax0_dir = '../noptmax0_testing'

In [ ]:
ens = pd.read_csv(os.path.join(wkdir,'{}.0.obs.csv'.format(pstroot)), index_col=0)

In [ ]:
phi = pd.read_csv(os.path.join(wkdir,'{}.phi.actual.csv'.format(pstroot)))

In [ ]:
parens = pd.read_csv(os.path.join(wkdir, '{}.0.par.csv'.format(pstroot)), index_col=0)

In [ ]:
parens

In [ ]:
phi.T.dropna().iloc[6:].hist(bins=50)

In [ ]:
phi

In [ ]:
# adjust threshold based on histogram
phi_threshold_for_keeping =  22500 # prior_mc_wide 
allphi=phi.T.dropna().iloc[6:].copy()
allphi.columns=['phi']
keepreals = allphi.loc[allphi.phi<phi_threshold_for_keeping].index
keepreals

In [ ]:
allphi.loc[keepreals].hist(bins=50)

In [ ]:
# truncate the ensemble to only keepreals (based on phi)
ens = ens.loc[keepreals]
# set percent_discrepancy to absolute value because we don't care about the sign
ens.perc_disc = ens.perc_disc.apply(lambda x: np.abs(x))

In [ ]:
# further truncate to eliminate bad mass balance runs 
print(len(ens))
ens = ens.loc[ens.perc_disc<0.01]
ens

In [ ]:
# reset keepreals based now both on phi and mass balance
keepreals = ens.index.values

In [ ]:
# look at phi again
allphi.loc[keepreals].hist(bins=50)

In [ ]:
allphi.loc[keepreals]

In [ ]:
pst = pyemu.Pst(os.path.join(wkdir,'{}.pst'.format(pstroot)))

In [ ]:
q50 = pyemu.helpers.calc_observation_ensemble_quantiles(ens, pst, [.5, .9])

In [ ]:
obs = pst.observation_data

In [ ]:
obs

In [ ]:
ens.to_csv(f'../notebooks_report/final_obs_ensembles/{pstroot}.prior.obs.ens.csv')

In [ ]:
with PdfPages(os.path.join(wkdir, 'postproc', '{}_obs_hist_plots.pdf'.format(pstroot))) as outpdf:
    for cob in obs.obsnme.values:
        plt.figure()
        ens[cob].hist(bins=50)
        obsval = obs.loc[cob].obsval
        plt.axvline(obsval, color='orange')
        plt.title(cob)
        outpdf.savefig()
        plt.close('all')

# trim the parameters to the acceptable PHI ensemble

# ONLY FOR MC WIDE

In [ ]:
parens.index = [str(i) for i in parens.index.values]
parens = parens.loc[keepreals]

In [ ]:
parens.to_csv(os.path.join(wkdir, 'never_iES_post_MC_parens.csv'))

In [ ]:
parens.describe()

In [ ]:
summary= parens.describe()

In [ ]:
summary.loc['mean'].loc[summary.loc['mean']>5]

In [ ]:
summary.loc['50%'].loc[summary.loc['50%']>100]

In [ ]:
summary.loc['75%'].loc[summary.loc['75%']>100]

In [ ]:
summary.loc['max'].loc[summary.loc['max']>100]

In [ ]:
np.unique(summary.T.index == pst.parameter_data.index)

In [ ]:
pst.parameter_data.parlbnd = summary.T['25%']
pst.parameter_data.parval1 = summary.T['50%']
pst.parameter_data.parubnd = summary.T['75%']

In [ ]:
pst.control_data.noptmax = 0

In [ ]:
pst.write(os.path.join(noptmax0_dir, 'never_iES_noptmax0.pst'))

In [ ]:
np.unique((pst.parameter_data.parubnd-pst.parameter_data.parlbnd)==0)

In [ ]:
pst.parameter_data.loc[(pst.parameter_data.parubnd-pst.parameter_data.parlbnd)==0]